<a href="https://colab.research.google.com/github/uwtintres/robotic_surgeries/blob/main/CRITIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project is the work of Alina Saduova, CSS graduate student at UWT, Spring'21. This notebook is used for the implementation of weight generation named CRITIC method. The calculation of the weights is done according to the input data that we provide, meaning according to the values of each alternative for each attribute. 

In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import statistics 

class Critic():
    evaluation_matrix = np.array([])  # Matrix
    '''
	Create an evaluation matrix consisting of m alternatives and n criteria.
	  '''

    def __init__(self, evaluation_matrix, criteria):
        # M×N matrix
        self.evaluation_matrix = np.array(evaluation_matrix, dtype="float")

        # M alternatives (options)
        self.row_size = len(self.evaluation_matrix)

        # N attributes/criteria
        self.column_size = len(self.evaluation_matrix[0])

        self.criteria = np.array(criteria, dtype="float")
    '''
	# Step 2
	The matrix is then normalised.
	'''

    def step_2(self):
        # normalized scores
        self.normalized_decision = np.copy(self.evaluation_matrix)
        for i in range(self.row_size):
            for j in range(self.column_size):
              if self.criteria[j]:
                worst = min(self.evaluation_matrix[:, j])
                best = max(self.evaluation_matrix[:, j])
              else:
                worst = max(self.evaluation_matrix[:, j])
                best = min(self.evaluation_matrix[:, j])
              #print("worst", worst)
              #print("best", best)
              self.normalized_decision[i,j] = (self.evaluation_matrix[i, j] - \
                                                   worst) / (best - worst)
        print("\nNormalized matrix:\n", self.normalized_decision)
              
    '''
	# Step 3
	Calculate the standard deviation and correlation matrix
	'''

    def step_3(self):
        self.stdev = np.empty(shape=(1, self.column_size))
        for j in range(self.column_size):
          self.stdev[0,j] = statistics.stdev(self.normalized_decision[:,j])
        #self.correlation_matrix = np.empty(shape=(self.col_size, self.col_size))
        self.correlation_matrix = np.corrcoef(self.normalized_decision.T)
        print("\nStandard deviation: ", self.stdev)
        print("\nCorrelation matrix:\n", self.correlation_matrix)
    '''
	# Step 4
	Calculate measure of the conflict matrix
	'''

    def step_4(self):
        ones_matrix = np.ones((self.column_size, self.column_size))
        self.conflict_matrix = np.subtract(ones_matrix, self.correlation_matrix)
        print("\nConflict matrix:\n", self.conflict_matrix)
        cj = np.empty(shape=(1, self.column_size))
        self.weights = np.empty(shape=(1, self.column_size))
        for i in range(self.column_size):
          cj[0, i] = self.stdev[0, i] * sum(self.conflict_matrix[i, :])
        print("\nCj: ", cj)
        for i in range(self.column_size):
          self.weights[0,i] = cj[0, i] / sum(cj[0, :])
        print("\nWeights:\n", self.weights)
        
    def calc(self):
        print("Step 1\nEvaluation matrix\n", self.evaluation_matrix, end="\n\n")
        self.step_2()
        #print("Step 2\nNormalized decision matrix\n", self.normalized_decision, end="\n\n")
        self.step_3()
        #print("Step 3\nPreference matrix\n", self.preference_matrix, end="\n\n")
        self.step_4()
        #print("Step 4\nOutranking flow\n", self.outranking_flow, end="\n\n")

Below is the driver code for the CRITIC method, where the input is the TOPSIS file with the data. The format of the excel is the same as for the TOPSIS itself. 

In [ ]:
import numpy as np
import pandas as pd 

filename = "TOPSIS with custom weights.xlsx"
sheetname = "Beginning"
xl = pd.read_excel(filename, sheet_name=sheetname)
xl = pd.read_excel(filename, sheet_name=sheetname, usecols=range(1, len(xl.columns)+1))

# initializing input matrix, weights, criterias
input_matrix = xl.tail(n=len(xl.index) - 2).values.tolist()
criterias = xl.head(n=0).columns.values
new_criterias = []
for criteria in criterias:
  # if the attribute is non beneficiary, meaning min value is better -> False, 
  # otherwise, the criteria is True
  new_criterias.append(False if criteria[0] == 'N' else True)
print('input matrix: ', input_matrix)
print('criterias: ', new_criterias)
# creating Topsis instance with the values above

input matrix:  [[1, 20, 100, 94], [4, 40, 30, 30], [5, 85, 90, 75], [7, 32, 600, 98], [3, 28, 800, 46], [8, 49, 1200, 15], [12, 56, 150, 68], [2, 71, 300, 23], [10, 66, 300, 57], [6, 79, 800, 83], [9, 10, 700, 39], [15, 18, 1000, 10]]
weights:  [0.6 0.2 0.1 0.1]
criterias:  [False, False, False, True]


In [ ]:
c = Critic(input_matrix, new_criterias)
# calculating the ranking
c.calc()


Step 1
Evaluation matrix
 [[1.0e+00 2.0e+01 1.0e+02 9.4e+01]
 [4.0e+00 4.0e+01 3.0e+01 3.0e+01]
 [5.0e+00 8.5e+01 9.0e+01 7.5e+01]
 [7.0e+00 3.2e+01 6.0e+02 9.8e+01]
 [3.0e+00 2.8e+01 8.0e+02 4.6e+01]
 [8.0e+00 4.9e+01 1.2e+03 1.5e+01]
 [1.2e+01 5.6e+01 1.5e+02 6.8e+01]
 [2.0e+00 7.1e+01 3.0e+02 2.3e+01]
 [1.0e+01 6.6e+01 3.0e+02 5.7e+01]
 [6.0e+00 7.9e+01 8.0e+02 8.3e+01]
 [9.0e+00 1.0e+01 7.0e+02 3.9e+01]
 [1.5e+01 1.8e+01 1.0e+03 1.0e+01]]

normalized [[ 1.          0.86666667  0.94017094  0.95454545]
 [ 0.78571429  0.6         1.          0.22727273]
 [ 0.71428571 -0.          0.94871795  0.73863636]
 [ 0.57142857  0.70666667  0.51282051  1.        ]
 [ 0.85714286  0.76        0.34188034  0.40909091]
 [ 0.5         0.48       -0.          0.05681818]
 [ 0.21428571  0.38666667  0.8974359   0.65909091]
 [ 0.92857143  0.18666667  0.76923077  0.14772727]
 [ 0.35714286  0.25333333  0.76923077  0.53409091]
 [ 0.64285714  0.08        0.34188034  0.82954545]
 [ 0.42857143  1.          0.42